In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import matplotlib

In [ ]:
from sympy import (symbols, simplify)
from sympy.physics.mechanics import dynamicsymbols, init_vprinting
from sympy.physics.mechanics import Lagrangian, ReferenceFrame, Point, Particle,inertia, RigidBody

In [ ]:
from optibot.symbolic import lagrange, diff_to_symb, SimpLagrangesMethod
from optibot.numpy import unpack

In [ ]:
from optibot.pseudospectral import (base_points, coll_points, matrix_D_bary, bary_poly, 
                                    LG_diff_end_p_fun, dynamic_error_pseudospectral, lagrangePolynomial,
                                    LG_diff_end_p_fun_cas)

In [ ]:
from functools import lru_cache

In [ ]:
import casadi as cas

In [ ]:
init_vprinting()

### Symbolic Problem Modelling

In [ ]:
def quaternion(theta, vect):
    vect = np.array(vect)
    mod = np.sqrt(np.sum(vect**2))
    vect = vect/mod
    quat = np.zeros(4)
    quat[0] = np.cos(theta/2)
    quat[1:] = np.sin(theta/2)*vect
    return quat

In [ ]:
quaternion(1, [1,2,0])

In [ ]:
_q = cas.SX.sym("q", 4)

In [ ]:
_W = cas.SX.zeros(3,4)
_W[0,0] = -_q[1]
_W[0,1] = _q[0]
_W[0,2] = _q[3]
_W[0,3] = -_q[2]
_W[1,0] = -_q[2]
_W[1,1] = -_q[3]
_W[1,2] = _q[0]
_W[1,3] = _q[1]
_W[2,0] = -_q[3]
_W[2,1] = _q[2]
_W[2,2] = -_q[1]
_W[2,3] = _q[0]
_W

In [ ]:
W_cas = cas.Function(
        "W",
        [_q],
        [_W,],
        ["q"],
        ["W"],
    )

In [ ]:
_R = cas.SX.zeros(3,3)
_R[0,0] = _q[0]**2 + _q[1]**2 - _q[2]**2 - _q[3]**2
_R[0,1] = 2 * (_q[1]*_q[2] - _q[0]*_q[3])
_R[0,2] = 2 * (_q[1]*_q[3] + _q[0]*_q[2])
_R[1,0] = 2 * (_q[1]*_q[2] + _q[0]*_q[3])
_R[1,1] = _q[0]**2 - _q[1]**2 + _q[2]**2 - _q[3]**2
_R[1,2] = 2 * (_q[2]*_q[3] - _q[0]*_q[1])
_R[2,0] = 2 * (_q[1]*_q[3] - _q[0]*_q[2])
_R[2,1] = 2 * (_q[3]*_q[2] + _q[0]*_q[1])
_R[2,2] = _q[0]**2 - _q[1]**2 - _q[2]**2 + _q[3]**2
_R

In [ ]:
_mod_q_sq = _q.T @ _q

In [ ]:
R_cas_no_unit = cas.Function(
        "R",
        [_q],
        [1 / _mod_q_sq * _R],
        ["q"],
        ["R"],
    )

In [ ]:
R_cas_no_unit(_q)

In [ ]:
_I_diag = cas.SX.eye(3)
_I_diag_vals = cas.SX.sym("I", 3)
_I_diag[0,0] = _I_diag_vals[0]
_I_diag[1,1] = _I_diag_vals[1]
_I_diag[2,2] = _I_diag_vals[2]

_I_diag

In [ ]:
I_cas_diag = cas.Function(
        "I",
        [_I_diag_vals],
        [_I_diag],
        ["I values"],
        ["I"],
    )

In [ ]:
_cross_mat = cas.SX(3,3)
_cross_mat[0,1] = -_I_diag_vals[2]
_cross_mat[0,2] = _I_diag_vals[1]
_cross_mat[1,0] = _I_diag_vals[2]
_cross_mat[1,2] = -_I_diag_vals[0]
_cross_mat[2,0] = -_I_diag_vals[1]
_cross_mat[2,1] = _I_diag_vals[0]
_cross_mat

In [ ]:
cross_mat_cas = cas.Function(
        "M_x",
        [_I_diag_vals],
        [_cross_mat],
        ["vector"],
        ["Cross product matrix"],
    )

In [ ]:
_q_dot = cas.SX.sym("q_d", 4)

In [ ]:
_omega = 2/_mod_q_sq*_W@_q_dot
_omega

In [ ]:
omega_cas = cas.Function(
        "w",
        [_q, _q_dot],
        [_omega],
        ["q", "q_d"],
        ["omega"],
    )

In [ ]:
_omega_expl = cas.SX.sym("w", 3)
_q_dot_omega = 1/2*_W.T @ _omega_expl

q_dot_omega_cas = cas.Function(
        "q_d",
        [_q, _omega_expl],
        [_q_dot_omega],
        ["q", "omega"],
        ["q_d"],
    )

In [ ]:
e3_x = cross_mat_cas([0,0,1])
e3 = cas.SX(3,1)
e3[2] = 1
_u = cas.SX.sym("u", 3)
_params = cas.SX.sym("param", 6)
_m, _g, _h, _Ix, _Iy, _Iz = cas.vertsplit(_params)
_I_diag_vals = _params[3:6]
_I_inv = cas.inv(I_cas_diag(_I_diag_vals))

In [ ]:
_omega_d = _I_inv @ (_u + _m*_g*_h* e3_x@R_cas_no_unit(_q).T@e3 - cross_mat_cas(_omega)@I_cas_diag(_I_diag_vals)@_omega)

In [ ]:
_omega_d.shape

In [ ]:
_q_d_d = 1/2*W_cas(_q_dot).T@_omega + 1/2*W_cas(_q).T@_omega_d

In [ ]:
_q_d_d.shape

In [ ]:
q_d_d_cas = cas.Function(
        "q_dot_dot",
        [_q, _q_dot, _u, _params],
        [_q_d_d],
        ["q", "q_d", "u", "params"],
        ["q_dot_dot"],
    )
q_d_d_cas.__name__ = q_d_d_cas.name()

In [ ]:
q_d_d_cas([0,0,0,1], [1,1,1,1], [0,1,1], [1,1,1,1,1,1]).shape

In [ ]:
def quat_to_cart(q):
    return params[2]*np.array(R_cas_no_unit(q))@np.array([0,0,1])

def quat_to_cart_arr(q_arr):
    l = q_arr.shape[0]
    res = np.zeros([l,3])
    for ii in range(l):
        res[ii,:] = quat_to_cart(q_arr[ii,:])
    return res


def omega_arr(q_arr, q_d_arr):
    l = q_arr.shape[0]
    res = np.zeros([l,3])
    for ii in range(l):
        res[ii,:] = omega_cas(q_arr[ii,:],q_d_arr[ii,:]).T
    return res

In [ ]:
def _energ(q, qdot, params):
    [m_n, g_n, h_n, Ix, Iy,Iz] = params 
    I =I_cas_diag([Ix, Iy, Iz])
    omeg = omega_cas(q, qdot)
    r_mat = R_cas_no_unit(q)
    i_iner = r_mat @ I @r_mat.T
    t = 0.5* omeg.T @i_iner @omeg
    z = quat_to_cart(q)[2]
    v = m_n * g_n * (-z)
    return (t+v), t, v

def energy_arr(q_arr, qdot_arr, params):
    l = q_arr.shape[0]
    res = np.zeros([l,3])
    for ii in range(l):
        _e = _energ(q_arr[ii,:], qdot_arr[ii,:], params)
        for jj in range(3):
            res[ii,jj] = _e[jj]
    return res

In [ ]:
m_n = 1
h_n = 1
g_n = 1
Ix = 1
Iy = 1
Iz = 0.5
params = (m_n, g_n, h_n, Ix, Iy,Iz)

In [ ]:
_energ(quaternion(0.5, [1,2,0]), [1,0,0,1], params)

## Numerico con Numba para que corra deprisa

In [ ]:
from numba import njit, prange

In [ ]:
@njit
def nquaternion(theta, vect):
    mod = np.sqrt(np.sum(vect**2))
    vect = vect/mod
    quat = np.zeros(4)
    quat[0] = np.cos(theta/2)
    quat[1:] = np.sin(theta/2)*vect
    return quat

In [ ]:
@njit
def W_num(_q):
    _W = np.zeros((3,4))
    _W[0,0] = -_q[1]
    _W[0,1] = _q[0]
    _W[0,2] = _q[3]
    _W[0,3] = -_q[2]
    _W[1,0] = -_q[2]
    _W[1,1] = -_q[3]
    _W[1,2] = _q[0]
    _W[1,3] = _q[1]
    _W[2,0] = -_q[3]
    _W[2,1] = _q[2]
    _W[2,2] = -_q[1]
    _W[2,3] = _q[0]
    return _W

In [ ]:
@njit
def R_num_no_unit(_q):
    _R = np.zeros((3,3))
    _R[0,0] = _q[0]**2 + _q[1]**2 - _q[2]**2 - _q[3]**2
    _R[0,1] = 2 * (_q[1]*_q[2] - _q[0]*_q[3])
    _R[0,2] = 2 * (_q[1]*_q[3] + _q[0]*_q[2])
    _R[1,0] = 2 * (_q[1]*_q[2] + _q[0]*_q[3])
    _R[1,1] = _q[0]**2 - _q[1]**2 + _q[2]**2 - _q[3]**2
    _R[1,2] = 2 * (_q[2]*_q[3] - _q[0]*_q[1])
    _R[2,0] = 2 * (_q[1]*_q[3] - _q[0]*_q[2])
    _R[2,1] = 2 * (_q[3]*_q[2] + _q[0]*_q[1])
    _R[2,2] = _q[0]**2 - _q[1]**2 - _q[2]**2 + _q[3]**2
    _mod_q_sq = _q.T @ _q
    return 1 / _mod_q_sq * _R

In [ ]:
@njit
def I_num_diag(_I_diag_vals):
    _I_diag = np.eye(3)
    _I_diag[0,0] = _I_diag_vals[0]
    _I_diag[1,1] = _I_diag_vals[1]
    _I_diag[2,2] = _I_diag_vals[2]

    return _I_diag

In [ ]:
@njit
def cross_mat_num(vect):
    _cross_mat = np.zeros((3,3))
    _cross_mat[0,1] = -vect[2]
    _cross_mat[0,2] = vect[1]
    _cross_mat[1,0] = vect[2]
    _cross_mat[1,2] = -vect[0]
    _cross_mat[2,0] = -vect[1]
    _cross_mat[2,1] = vect[0]

    return _cross_mat

In [ ]:
@njit
def omega_num(_q, _q_dot):
    _mod_q_sq = _q.T @ _q
    _W = W_num(_q)
    _omega = 2/_mod_q_sq*_W@_q_dot
    return _omega

@njit
def q_dot_omega_num(_q, _omega_expl):
    _W = W_num(_q)
    _q_dot_omega = 1/2*_W.T @ _omega_expl
    return _q_dot_omega

In [ ]:
@njit
def omega_d_num(_q, _q_dot, _u, _params):
    e3 = np.array((0,0,1.))
    e3_x = cross_mat_num(e3)
    _m, _g, _h, _Ix, _Iy, _Iz = _params
    _I_diag_vals = np.array(_params[3:6])
    _I = I_num_diag(_I_diag_vals)
    _I_inv = np.linalg.inv(_I)
    _omega = omega_num(_q, _q_dot)
    
    _t1 = _u + _m*_g*_h* e3_x@R_num_no_unit(_q).T@e3
    _t3 = - cross_mat_num(_omega)@_I@_omega
    _omega_d = _I_inv @ (_t1 + _t3)
    
    return _omega_d

@njit
def q_d_d_num(_q, _q_dot, _u, _params):
    _omega_d = omega_d_num(_q, _q_dot, _u, _params)
    _omega = omega_num(_q, _q_dot)
    
    _q_d_d = 1/2*W_num(_q_dot).T@_omega + 1/2*W_num(_q).T@_omega_d
    
    return _q_d_d

In [ ]:
@njit
def quat_to_cart_n(q, params):
    return params[2]*R_num_no_unit(q)@np.array([0.,0,1])

@njit(parallel = True)
def quat_to_cart_arr(q_arr, params):
    l = q_arr.shape[0]
    res = np.zeros((l,3))
    for ii in prange(l):
        res[ii,:] = quat_to_cart_n(q_arr[ii,:], params)
    return res


@njit(parallel = True)
def omega_arr(q_arr, q_d_arr):
    l = q_arr.shape[0]
    res = np.zeros((l,3))
    for ii in prange(l):
        res[ii,:] = omega_num(q_arr[ii,:],q_d_arr[ii,:])
    return res

In [ ]:
@njit
def _energ(q, qdot, params):
    (m_n, g_n, h_n, Ix, Iy,Iz) = params 
    I =I_num_diag((Ix, Iy, Iz))
    omeg = omega_num(q, qdot)
    r_mat = R_num_no_unit(q)
    t = 0.5* omeg.T @I @omeg
    z = quat_to_cart_n(q, params)[2]
    v = m_n * g_n * (-z)
    return (t+v), t, v

@njit(parallel = True)
def energy_arr(q_arr, qdot_arr, params):
    l = q_arr.shape[0]
    res = np.zeros((l,3))
    for ii in prange(l):
        _e = _energ(q_arr[ii,:], qdot_arr[ii,:], params)
        for jj in prange(3):
            res[ii,jj] = _e[jj]
    return res

## Integración a las bravas

In [ ]:
from optibot.schemes import expand_G

In [ ]:
x_d_cas = expand_G(q_d_d_cas, 'casadi')
def embudo(x, u, p):
    sol = x_d_cas(x,u,p)
    sol = np.array(sol).flatten()
    return sol


@njit
def x_d_num(x, u, params):
    q = x[:4]
    v = x[4:]
    qdd = q_d_d_num(q, v, u, params)
    xd = np.zeros(8)
    xd[:4] = v
    xd[4:] = qdd
    return xd

In [ ]:
x0 = np.zeros(8)
th0 = 0
q0 = quaternion(th0, [1,0,0])
qd0 = q_dot_omega_cas(q0, [1.5,0,1])
x0[:4] = q0
x0[4:] = qd0.T

In [ ]:
x_d_num(x0, np.zeros(3), params)

In [ ]:
x_d_cas(x0, np.zeros(3), params)

In [ ]:
embudo(x0, [0,1,1],params)

In [ ]:
from optibot.schemes import (integrate_trapz, integrate_trapz_mod, integrate_rk4, integrate_hs, 
                             integrate_hs_mod, interpolated_array,
                             coherent_dimensions, trapz_step, rk4_step, hs_step, vec_len)
from scipy.interpolate import interp1d

In [ ]:
def energy_fluctuations(q_arr, v_arr, t_arr, params):
    t_e = energy_arr(q_arr, v_arr, params)[:,0]
    dE = np.max(t_e) - np.min(t_e)
    
    _t_e = t_e[1:]-t_e[:-1]
    _t_e_1 = np.sign(_t_e)
    _t_e_2 = _t_e_1[1:] - _t_e_1[:-1]

    _maxis = []
    _minis = []
    for jj in range(len(_t_e_2)):
        if _t_e_2[jj] > 1:
            _minis.append(jj+1)
        elif _t_e_2[jj] < -1:
            _maxis.append(jj+1)
    
    _max_x = []
    _max_y = []
    _min_x = []
    _min_y = []

    for ii in _maxis:
        _max_x.append(t_arr[ii])
        _max_y.append(t_e[ii])
    for ii in _minis:
        _min_x.append(t_arr[ii])
        _min_y.append(t_e[ii])
            
    if len(_maxis)<3 or len(_minis)<3:
        maxinterp = None
        mininterp = None
        mean_amplitude = None
        tendency = None
        d_tendency = None
    else:
        

        maxinterp = interp1d(_max_x, _max_y, kind = 'cubic', fill_value='extrapolate')
        mininterp = interp1d(_min_x, _min_y, kind = 'cubic', fill_value='extrapolate')
        tendency = (maxinterp(t_arr)+mininterp(t_arr))/2
        mean_amplitude = np.mean(maxinterp(t_arr)-mininterp(t_arr))
        d_tendency = np.max(tendency) - np.min(tendency)
    
    data = {
        'total_energy':t_e,
        'delta_energy':dE,
        'max_interp':maxinterp,
        'min_interp':mininterp,
        'tendency':tendency,
        'delta_tendency': d_tendency,
        'mean_amplitude':mean_amplitude,
        'max_x':_max_x,
        'max_y':_max_y,
        'min_x':_min_x,
        'min_y':_min_y
    }
    return data

In [ ]:
def free_pendulum(x0, params, n, dt, integrator_f):
    x_arr = integrator_f(
        x_0=x0,
        u = np.zeros([n,3]),
        F = x_d_num,
        dt = dt,
        params=params
    )
    q_arr = x_arr[:,:4]
    v_arr = x_arr[:,4:]
    t_arr = np.linspace(0,n*dt,n+1)
    energ_data = energy_fluctuations(q_arr, v_arr, t_arr, params)
    data = {
        'q':q_arr,
        'v':v_arr,
        't':t_arr,
        'e':energ_data,
    }
    return data

In [ ]:
x_arr = integrate_trapz(
    x_0=x0,
    u = np.zeros([10000,3]),
    F = x_d_num,
    dt = 0.01,
    params=params
)
q_arr = x_arr[:,:4]
v_arr = x_arr[:,4:]

t_interp = np.linspace(0,2,250)
x_interp, u_interp = interpolated_array(
    x_arr,
    np.zeros([10000,3]),
    0.1,
    t_interp,
    params,
    F=embudo,
    X_dot=None,
    scheme="trapz",
    u_scheme="lin",
    scheme_params={},
)
q_interp = x_interp[:,:4]
plt.figure(figsize=[14,6])
plt.plot(np.linspace(0,100,10001),np.sum(q_arr**2, axis = 1), 'o' )
plt.plot(t_interp, np.sum(q_interp**2, axis = 1))
plt.grid()

t_interp = np.linspace(0,2,250)
x_interp, u_interp = interpolated_array(
    x_arr,
    np.zeros([20,3]),
    0.1,
    t_interp,
    params,
    F=embudo,
    X_dot=None,
    scheme="trapz",
    u_scheme="lin",
    scheme_params={},
)
q_interp = x_interp[:,:4]
plt.figure(figsize=[14,6])
plt.plot(np.linspace(0,2,21),q_arr, 'o' )
plt.plot(t_interp, q_interp)
plt.grid()

In [ ]:
pend_trapz = free_pendulum(x0, params, 10000, 0.01, integrate_trapz)
t_arr = pend_trapz['t']
_data = pend_trapz['e']

In [ ]:
plt.figure(figsize=[14,6])
plt.plot(t_arr,_data['total_energy'], marker = '')
plt.plot(_data['max_x'], _data['max_y'], 'o')
plt.plot(_data['min_x'], _data['min_y'], 'o')
plt.plot(t_arr,_data['max_interp'](t_arr))
plt.plot(t_arr,_data['min_interp'](t_arr))
plt.plot(t_arr,_data['tendency'])
plt.grid()

In [ ]:
plt.figure(figsize=[14,6])
q_arr = pend_trapz['q']
v_arr = pend_trapz['v']
plt.plot(np.linspace(0,100,10001),omega_arr(q_arr, v_arr)[:,:], marker = '')
plt.grid()
#_w_m = (2*m_n*g_n*h_n*(1-np.cos(th0))/Ix)**0.5
#plt.hlines(_w_m, 0, 100, linestyles='dashed')

In [ ]:
plt.figure(figsize=[14,6])
cart_arr = quat_to_cart_arr(q_arr, np.array(params))
plt.plot(t_arr,cart_arr, marker = '')
plt.grid()

In [ ]:
%matplotlib notebook

In [ ]:
ax = plt.axes(projection='3d')

ax.plot3D(cart_arr[:,0], cart_arr[:,1], cart_arr[:,2], 'gray', marker = '')

In [ ]:
%matplotlib inline

## Metodos de integración y error de energía

In [ ]:
scheme_data = {
    'trapz':{
        'name':'trapezoidal',
        'integ_f': integrate_trapz
    },
    'trapz_mod':{
        'name':'trapezoidal second order',
        'integ_f': integrate_trapz_mod
    },
    'rk4':{
        'name':'Runge Kutta 4',
        'integ_f': integrate_rk4
    },
    'hs':{
        'name':'Hermite Simpson',
        'integ_f': integrate_hs
    },
    'hs_mod':{
        'name':'Hermite Simpson Second Order',
        'integ_f': integrate_hs_mod
    },
}

In [ ]:
schemes_qqdot = ['trapz', 'trapz_mod', 'rk4', 'hs', 'hs_mod']
n = 10000
dt = 0.01

results_qqdot = {}

for scheme in schemes_qqdot:
    print('calculating scheme', scheme_data[scheme]['name'])
    integ_f = scheme_data[scheme]['integ_f']
    results_qqdot[scheme] = free_pendulum(x0, params, n, dt, integ_f)

In [ ]:
for scheme in schemes_qqdot:
    print(scheme_data[scheme]['name'])
    print('\tDelta energy:', results_qqdot[scheme]['e']['delta_energy'])
    print('\tDelta tendency:', results_qqdot[scheme]['e']['delta_tendency'])

In [ ]:

plt.figure(figsize=[14,10])
for scheme in schemes_qqdot:
    t_arr = results_qqdot[scheme]['t']
    en_arr = results_qqdot[scheme]['e']['total_energy']
    en_diff = np.abs(en_arr-en_arr[0])
    plt.plot(t_arr, en_diff, label = scheme_data[scheme]['name'])
plt.legend()
plt.grid()
plt.ylim(1e-13, 1e-1)
plt.yscale('log')

In [ ]:

plt.figure(figsize=[14,10])
for scheme in schemes_qqdot:
    t_arr = results_qqdot[scheme]['t']
    if results_qqdot[scheme]['e']['tendency'] is None:
        en_arr = results_qqdot[scheme]['e']['total_energy']
    else:
        en_arr = results_qqdot[scheme]['e']['tendency']
    en_diff = np.abs(en_arr-en_arr[0])
    plt.plot(t_arr, en_diff, label = scheme_data[scheme]['name'])
plt.legend()
plt.grid()
plt.ylim(1e-13, 1e-1)
plt.yscale('log')

## interpolaciones y desviación

In [ ]:
x_arr = integrate_rk4(
    x_0=x0,
    u = np.zeros([20,3]),
    F = x_d_num,
    dt = 0.1,
    params=params
)
q_arr = x_arr[:,:4]
v_arr = x_arr[:,4:]

In [ ]:
t_interp = np.linspace(0,2,250)
x_interp, u_interp = interpolated_array(
    x_arr,
    np.zeros([20,3]),
    0.1,
    t_interp,
    params,
    F=embudo,
    X_dot=None,
    scheme="hs_scipy",
    u_scheme="lin",
    scheme_params={},
)
q_interp = x_interp[:,:4]
plt.figure(figsize=[14,6])
plt.plot(np.linspace(0,2,21),np.sum(q_arr**2, axis = 1), 'o' )
plt.plot(t_interp, np.sum(q_interp**2, axis = 1))
plt.grid()

In [ ]:
x_arr = integrate_hs(
    x_0=x0,
    u = np.zeros([20,3]),
    F = x_d_num,
    dt = 0.1,
    params=params
)
q_arr = x_arr[:,:4]
v_arr = x_arr[:,4:]

In [ ]:
t_interp = np.linspace(0,2,250)
x_interp, u_interp = interpolated_array(
    x_arr,
    np.zeros([20,3]),
    0.1,
    t_interp,
    params,
    F=x_d_num,
    X_dot=None,
    scheme="hs",
    u_scheme="lin",
    scheme_params={},
)
q_interp = x_interp[:,:4]
plt.figure(figsize=[14,6])
plt.plot(np.linspace(0,2,21),np.sum(q_arr**2, axis = 1), 'o' )
plt.plot(t_interp, np.sum(q_interp**2, axis = 1))
plt.grid()

## Normalizando a cada paso

In [ ]:
def normalize_x(x):
    x = np.array(x)
    q = x[:4]
    mod = np.sqrt(np.sum(q**2))
    q = q/mod
    x[:4] = q
    return x

In [ ]:
@coherent_dimensions
def integrate_trapz_normalized(x_0, u, F, dt, params):
    x = [
        x_0,
    ]
    for ii in range(0, vec_len(u) - 1):
        x_i = trapz_step(x[-1], u[ii], u[ii + 1], F, dt, params)
        x_i = normalize_x(x_i)
        x.append(x_i)
    x_i = trapz_step(x[-1], u[-1], u[-1], F, dt, params)
    x_i = normalize_x(x_i)
    x.append(x_i)
    return np.array(x)

x_arr = integrate_trapz_normalized(
    x_0=x0,
    u = np.zeros([20,3]),
    F = x_d_num,
    dt = 0.1,
    params=params
)
q_arr = x_arr[:,:4]
v_arr = x_arr[:,4:]

In [ ]:
t_interp = np.linspace(0,2,250)
x_interp, u_interp = interpolated_array(
    x_arr,
    np.zeros([20,3]),
    0.1,
    t_interp,
    params,
    F=x_d_num,
    X_dot=None,
    scheme="trapz",
    u_scheme="lin",
    scheme_params={},
)
q_interp = x_interp[:,:4]
plt.figure(figsize=[14,6])
plt.plot(np.linspace(0,2,21),np.sum(q_arr**2, axis = 1), 'o' )
plt.plot(t_interp, np.sum(q_interp**2, axis = 1))
plt.grid()

In [ ]:
@coherent_dimensions
def integrate_rk4_normalized(x_0, u, F, dt, params):
    x = [
        x_0,
    ]
    for ii in range(vec_len(u)):
        x_i = rk4_step(x[-1], u[ii], F, dt, params)
        x_i = normalize_x(x_i)
        x.append(x_i)
    return np.array(x)

x_arr = integrate_rk4_normalized(
    x_0=x0,
    u = np.zeros([20,3]),
    F = x_d_num,
    dt = 0.1,
    params=params
)
q_arr = x_arr[:,:4]
v_arr = x_arr[:,4:]

In [ ]:
t_interp = np.linspace(0,2,400)
x_interp, u_interp = interpolated_array(
    x_arr,
    np.zeros([20,3]),
    0.1,
    t_interp,
    params,
    F=x_d_num,
    X_dot=None,
    scheme="hs",
    u_scheme="lin",
    scheme_params={},
)
q_interp = x_interp[:,:4]
plt.figure(figsize=[14,6])
plt.plot(np.linspace(0,2,21),np.sum(q_arr**2, axis = 1), 'o' )
plt.plot(t_interp, np.sum(q_interp**2, axis = 1))
plt.grid()

In [ ]:
@coherent_dimensions
def integrate_hs_normalized(x_0, u, F, dt, params):
    x = [
        x_0,
    ]
    for ii in range(0, vec_len(u) - 1):
        x_i = hs_step(x[-1], u[ii], u[ii + 1], F, dt, params)
        x_i = normalize_x(x_i)
        x.append(x_i)
    x_i = hs_step(x[-1], u[-1], u[-1], F, dt, params)
    x_i = normalize_x(x_i)
    x.append(x_i)
    return np.array(x)


x_arr = integrate_hs_normalized(
    x_0=x0,
    u = np.zeros([20,3]),
    F = x_d_num,
    dt = 0.1,
    params=params
)
q_arr = x_arr[:,:4]
v_arr = x_arr[:,4:]

In [ ]:
t_interp = np.linspace(0,2,400)
x_interp, u_interp = interpolated_array(
    x_arr,
    np.zeros([20,3]),
    0.1,
    t_interp,
    params,
    F=x_d_num,
    X_dot=None,
    scheme="hs",
    u_scheme="lin",
    scheme_params={},
)
q_interp = x_interp[:,:4]
plt.figure(figsize=[14,6])
plt.plot(np.linspace(0,2,21),np.sum(q_arr**2, axis = 1), 'o' )
plt.plot(t_interp, np.sum(q_interp**2, axis = 1))
plt.grid()

## New opti

In [ ]:
from optibot.opti import Opti_Problem
import time

def sph_prob(scheme, N, ini_guess = 'lin', solve_repetitions = 1, t_end = 2, silent = False, ):
    _opti = Opti_Problem(
        LM=q_d_d_cas,
        params = params,
        scheme = scheme,
        ini_guess= ini_guess,
        t_end = t_end,
        silent= silent,
    )
    _opti.dynamic_setup()
    _opti.opti_setup(N)
    _opti.initial_guess([0,0], [1, np.pi])
    _opti.u_sq_cost()
    _opti.apply_scheme()
    
    X_s = _opti.opti_points['x_s']
    X_e = _opti.opti_points['x_e']

    _opti.opti.subject_to(X_s.T == [0,0,0,0])
    _opti.opti.subject_to(X_e.T == [1, np.pi,0,0])
    _opti.chrono_solve(solve_repetitions=solve_repetitions,)
    return _opti.results

_opti = Opti_Problem(
        LM=q_d_d_cas,
        params = params,
        scheme = 'trapz_mod',
        ini_guess= 'lin',
        t_end = 0.1,
        silent= False,
        verbose = True
    )

_opti.dynamic_setup(func_kind='g_q', n_q = 4, n_u = 3)
_opti.opti_setup(20)

q0 = quaternion(1, [1,2,0])
q1 = quaternion(1, [1,0,2])
_opti.initial_guess(q0,q1)

_opti.u_sq_cost()
_opti.apply_scheme()

q_s = _opti.opti_points['q_s']
v_s = _opti.opti_points['v_s']

_opti.opti.subject_to(q_s.T == q0)
_opti.opti.subject_to(v_s.T == 0)


q_e = _opti.opti_points['q_e']
v_e = _opti.opti_points['v_e']

_opti.opti.subject_to(q_e.T == q1)
_opti.opti.subject_to(v_e.T == 0)

U = _opti.opti_arrs['u']
#_opti.opti.subject_to(U == 0.5)

_opti.chrono_solve(solve_repetitions=1,)

plt.plot(_opti.opti.debug.value(_opti.opti_arrs['q']))

plt.plot(_opti.opti.debug.value(_opti.opti_arrs['u']))

## con c variable

In [ ]:
_u_cvar = cas.SX.sym("u", 4)
_omega_d_cvar = _I_inv @ (_u_cvar[0:3] + _m*_g*_h* e3_x@R_cas_no_unit(_q).T@e3 - cross_mat_cas(_omega)@I_cas_diag(_I_diag_vals).T@_omega)

In [ ]:
_q_d_d_cvar = 1/2*W_cas(_q_dot).T@_omega + 1/2*W_cas(_q).T@_omega_d_cvar + _u_cvar[3]*_q_dot

In [ ]:
q_d_d_cas_cvar = cas.Function(
        "q_dot_dot",
        [_q, _q_dot, _u_cvar, _params],
        [_q_d_d_cvar],
        ["q", "q_d", "u", "params"],
        ["q_dot_dot"],
    )
q_d_d_cas_cvar.__name__ = q_d_d_cas_cvar.name()

In [ ]:
N_step = 50
t_end = 5
_opti = Opti_Problem(
        LM=q_d_d_cas_cvar,
        params = params,
        scheme = 'hs_mod',
        ini_guess= 'lin',
        t_end = t_end,
        silent= False,
        verbose = True
    )

_opti.dynamic_setup(func_kind='g_q', n_q = 4, n_u = 4)
_opti.opti_setup(N_step)

q0 = quaternion(1, [1,2,0])
q1 = quaternion(1, [1,2,0])
_opti.initial_guess(q0,q1)

#_opti.u_sq_cost()

_opti.apply_scheme()


U = _opti.opti_arrs['u']
_opti.opti.subject_to(U[:,:3] == 0)

q_s = _opti.opti_points['q_s']
v_s = _opti.opti_points['v_s']
q_arr_cas = _opti.opti_arrs['q']

_opti.opti.subject_to(q_s.T == q0)
_opti.opti.subject_to(v_s.T == 0)

_opti.opti.subject_to(cas.sum2(q_arr_cas[1:,:]**2) == 1)

_opti.chrono_solve(solve_repetitions=1,)

In [ ]:
%matplotlib inline

In [ ]:
plt.figure(figsize=[14,6])
plt.plot(_opti.results['u'][:,-1])
plt.grid()

In [ ]:
plt.plot(_opti.results['u'][:,:-1])

In [ ]:
plt.figure(figsize=[14,6])
plt.plot(_opti.results['q'], marker = 'o')

In [ ]:
plt.figure(figsize=[14,6])
plt.plot(_opti.results['v'], marker = 'o')

In [ ]:
plt.plot(np.sum(_opti.results['q']**2, axis=1), marker = 'o')

In [ ]:
x_d_cas_cvar = expand_G(q_d_d_cas_cvar, 'casadi')
def embudo_cvar(x, u, p):
    sol = x_d_cas_cvar(x,u,p)
    sol = np.array(sol).flatten()
    return sol

In [ ]:
nn = _opti.results['x'].shape[0]
t_interp = np.linspace(0,2,500)
x_interp, u_interp = interpolated_array(
    _opti.results['x'],
    _opti.results['u'],
    2/(nn-1),
    t_interp,
    params,
    F=embudo_cvar,
    X_dot=None,
    scheme="hs",
    u_scheme="lin",
    scheme_params={},
)
q_interp = x_interp[:,:4]
plt.figure(figsize=[14,6])
plt.plot(np.linspace(0,2,nn),np.sum(_opti.results['q']**2, axis = 1), 'o' )
plt.plot(t_interp, np.sum(q_interp**2, axis = 1))
plt.grid()

In [ ]:
plt.figure(figsize=[14,6])
q_arr = _opti.results['q']
v_arr = _opti.results['v']
cart_arr = quat_to_cart_arr(q_arr, params)
plt.plot(np.linspace(0,t_end,51),cart_arr, marker='o')
plt.grid()

In [ ]:
plt.figure(figsize=[14,6])
plt.plot(np.linspace(0,t_end,51),energy_arr(q_arr, v_arr, params)[:,:], marker = '')
plt.grid()

In [ ]:
%matplotlib notebook

In [ ]:
plt.figure(figsize=[10,10])
ax = plt.axes(projection='3d')

ax.plot3D(cart_arr[:,0], cart_arr[:,1], cart_arr[:,2], 'gray', marker = 'o')

In [ ]:
%matplotlib inline